In [9]:
import pandas as pd
import sklearn as sklearn
import scipy.stats as stats
from sklearn.svm import SVC 
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import RandomizedSearchCV

from utils.base_set import X_train, y_train, seed

### SVM

In [10]:
model = SVC(random_state=seed)
k = 5
cv = StratifiedKFold(n_splits=k, shuffle=True, random_state=seed)
params = {
    'C': stats.uniform(1e-16, 1), 
    'gamma': ['scale', 'auto'],
    'kernel': ["rbf", "linear", "poly", "sigmoid"],
    'degree': stats.randint(2, 10)
}
n = 100
classifier_SVM = RandomizedSearchCV(estimator=model, 
                                param_distributions=params,
                                cv=k,
                                n_iter=n,
                                scoring="roc_auc", 
                                random_state=seed)
classifier_SVM.fit(X_train, y_train)

RandomizedSearchCV(cv=5, estimator=SVC(random_state=8241), n_iter=100,
                   param_distributions={'C': <scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x7bda78c1f220>,
                                        'degree': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x7bda78b6d4b0>,
                                        'gamma': ['scale', 'auto'],
                                        'kernel': ['rbf', 'linear', 'poly',
                                                   'sigmoid']},
                   random_state=8241, scoring='roc_auc')

In [13]:
best = classifier_SVM.best_params_
auc_roc = classifier_SVM.best_score_
print(best)
print(auc_roc)

{'C': 0.9010665799255806, 'degree': 8, 'gamma': 'scale', 'kernel': 'rbf'}
0.8734749513510979


In [15]:
columns_to_keep = [f"param_{param}" for param in params.keys()] + ['mean_test_score','rank_test_score']
pd.DataFrame(classifier_SVM.cv_results_).sort_values('rank_test_score')[columns_to_keep]

,param_C,param_gamma,param_kernel,param_degree,mean_test_score,rank_test_score
80,0.901067,scale,rbf,8,0.873475,1
60,0.875746,scale,rbf,3,0.872442,2
71,0.79047,scale,rbf,6,0.869464,3
37,0.772139,scale,rbf,7,0.869121,4
4,0.774049,scale,rbf,6,0.869121,4
...,...,...,...,...,...,...
40,0.842751,auto,sigmoid,7,0.625289,96
81,0.911703,auto,sigmoid,7,0.624806,97
44,0.932612,auto,sigmoid,2,0.615937,98
67,0.717453,scale,poly,8,0.611388,99
